In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from pprint import pprint
import copy

# 1. Загрузить датасет

In [ ]:
# https://www.kaggle.com/competitions/playground-series-s3e11/data
dataset_df = pd.read_csv('dataset/train.csv', index_col="id")
dataset_df.sample(7)

Описание свойств:
* store_sales(in millions) — продажи в магазине (в миллионах долларов)
* unit_sales(in millions) — продажи в магазине (в миллионах) Количество
* total_children — ВСЕ ДЕТИ В ДОМЕ
* num_children_at_home — количество детей в доме согласно заполненным клиентами данным
* avg_cars_at home(approx).1 — среднее количество автомобилей в доме (приблизительно)
* gross_weight — вес брутто
* recyclable_package — ПИЩЕВОЙ ПРОДУКТ В ПЕРЕРАБАТЫВАЕМОЙ_УПАКОВКЕ
* low_fat — ПИЩЕВОЙ ПРОДУКТ С НИЗКИМ СОДЕРЖАНИЕМ ЖИРА
* units_per_case — ЕДИНИЦ/ЯЩИК, ДОСТУПНЫХ НА КАЖДОЙ ПОЛКЕ В МАГАЗИНЕ
* store_sqft — ПЛОЩАДЬ МАГАЗИНА, ДОСТУПНАЯ В КВАДРАТНЫХ ФУТАХ
* coffee_bar — КОФЕЙНЫЙ БАР, доступный в магазине
* video_store — ВИДЕОМАГАЗИН/игровой магазин, доступный в магазине
* salad_bar — САЛАТНЫЙ БАР, доступный в магазине
* prepared_food — ПРИГОТОВЛЕННАЯ ЕДА, доступная в магазине
* florist — цветочные полки, доступные в магазине
* cost — затраты на привлечение клиентов в долларах

# 2. EDA датасета

In [ ]:
def eda_df(df):
    """Провести EDA для датафрейма"""
    df_describe = df.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])
    # посчитать долю пропусков
    df_describe.loc["%nan"] = (df.isna().mean()*100).to_list()
    # посчитать дисперсию
    columns_var = []
    for column in df_describe.columns:
        columns_var.append(df[column].var())
    df_describe.loc['var'] = columns_var
    return df_describe

In [ ]:
dataset_df.info()

In [ ]:
dataset_df.sample(5)

In [ ]:
dataset_df_describe = eda_df(dataset_df)
dataset_df_describe

In [ ]:
numerical = []
categorical = []
for col in dataset_df.columns:
    if dataset_df[col].nunique() <= 2:
        categorical.append(col)
    else:
        numerical.append(col)
pprint(f'categorical={categorical}')
pprint(f'numerical={numerical}')

Выводы по набору данных:
  * все переменные числовые
  * пропусков нет

# 3. Подготовка датасета

In [ ]:
import missingno as msno
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def show_boxes(df, columns, ncols = 3):
    """Показать 'ящики_с_усами' для набора df.
    Ящики будут показаны для столбцов датафрема, перечисленных в columns.
    Графики будут показаны в несколько столбцов, количество которых задается в параметре ncols."""
    nrows = int(round((len(columns) + 0.5) / ncols, 0))
    nrows = nrows if nrows > 1 else 1

    fig = make_subplots(rows=nrows, cols=ncols)
    fig.update_layout(
        title_x=0.5,
        title_text="Ящики с усами",
        height=500*nrows, 
        width=800
    )
    i = 0
    for r in range(nrows):
        for c in range(ncols):
            fig.add_box(y=df[columns[i]], name=columns[i], row=r+1, col=c+1)
            i += 1
            if i >= len(columns):
                break
        if i >= len(columns):
            break
    fig.show()     

## 3.а. Анализ целевой переменной cost

In [ ]:
fig = make_subplots(rows=1, cols=2)
fig.update_layout(
    title_x=0.5,
    title_text="Анализ целевой переменной cost",
    height=500, 
    width=1200
)
fig.add_box(y=dataset_df["cost"], name="cost", row=1, col=1)
fig.add_histogram(x=dataset_df["cost"], name="cost", row=1, col=2)
fig.show()   
    

Распределение целевой переменной - равномерное.
Выбросов нет.

## 3.b. Обработка пропусков

In [ ]:
_ = msno.matrix(dataset_df, figsize=(12,2), fontsize=9)

Пропусков нет

## 3.c. Обработка выбросов

In [ ]:
columns = dataset_df.columns.to_list()
columns.remove("cost")
show_boxes(dataset_df, columns)

Выбросы есть в полях: `store_sales(in millions)`, `unit_sales(in millions)`, `num_children_at_home`. 
Но `unit_sales(in millions)`, `num_children_at_home` это скорее ранговые переменные.  Проверим их распределение

In [ ]:
fig = make_subplots(rows=1, cols=2)
fig.update_layout(
    title_x=0.5,
    title_text="Распределение значений свойств",
    height=500, 
    width=1200
)
fig.add_histogram(x=dataset_df["unit_sales(in millions)"], name="unit_sales(in millions)", row=1, col=1)
fig.add_histogram(x=dataset_df["num_children_at_home"], name="num_children_at_home", row=1, col=2)
fig.show()   

У `unit_sales(in millions)` - нормальное распределение, у `num_children_at_home` - экспоненциальное.

Предполагаю, что выбросы специально обрабатывать не надо у этих свойств.

Уберем выбросы у `store_sales(in millions)`. Т.к. выбросы все "сверху", то заменим их максимальные значения внутри IQR.

In [ ]:
column = 'store_sales(in millions)'
Q3 = np.quantile(dataset_df[column], 0.75, axis=0)
Q1 = np.quantile(dataset_df[column], 0.25, axis=0)
IQR = Q3 - Q1
upper = Q3 + 1.5 * IQR
lower = Q1 - 1.5 * IQR
dataset_df.loc[dataset_df[column] > upper]
# добавим столбец, в котором избавимся от выбросов в 'store_sales(in millions)', заменив их максимальными или минимальными значениями
dataset_df[f'{column}_without_outliers'] = dataset_df[column].map(lambda x: lower if x<lower else x if x<=upper else upper)


In [ ]:
dataset_df.sample(7)

In [ ]:
# проверить, что в новом столбце нет выбросов
show_boxes(dataset_df, ['store_sales(in millions)', 'store_sales(in millions)_without_outliers'], ncols=2)

## 3.d. Анализ корреляцие между столбцами

In [ ]:
fig = px.imshow(dataset_df.corr(method='spearman'), height=1000, width=1000, text_auto='.2f')
fig.show()

Высоковата корреляция между столбцами:
  * florist + coffer_bar
  * florist + video_store
  * florist + salad_bar
  * florist + prepared_food
  * prepared_food + video_store
  * salad_bar + video_store
  * salad_bar + prepared_food
  * video_store + coffer_bar
  * store_sales(in millions) + unit_sales(in millions)
  * total_children и num_children_at_home.

Попробуем исключить из модели florist, video_store, salad_bar, store_sales(in millions) и total_children

In [ ]:

columns_for_remove = ['florist', 'video_store', 'unit_sales(in millions)']
columns_without_corr = dataset_df.columns.to_list()
for col in columns_for_remove:
    columns_without_corr.remove(col)
fig = px.imshow(dataset_df[columns_without_corr].corr(method='spearman'), height=1000, width=1000, text_auto='.2f')
fig.show()

## 3.e. Разделить на набор данных на обучающую и тестовую выборки

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Разделить набор данных на тренировочную и тестовую выборки
columns = dataset_df.columns.to_list()
# убрать целевую переменную
columns.remove("cost")
# убать оригинальный столбек с выбросами
columns.remove('store_sales(in millions)')
X_train, X_test, y_train, y_test = train_test_split(dataset_df[columns], 
                                                    dataset_df['cost'],
                                                    test_size=0.2, random_state=42)
print(f'X_train.shape={X_train.shape},  X_test.shape={X_test.shape}')
print(f'y_train.mean()={y_train.mean()}, y_test.mean()={y_test.mean()}')


In [ ]:
# Разделить набор данных на тренировочную и тестовую выборки (без столбцов с корреляцией)
columns = copy.deepcopy(columns_without_corr)
# убрать целевую переменную
columns.remove("cost")
# убать оригинальный столбек с выбросами
columns.remove('store_sales(in millions)')
X_train_wo_corr, X_test_wo_corr, y_train_wo_corr, y_test_wo_corr = train_test_split(dataset_df[columns], 
                                                                                    dataset_df['cost'],
                                                                                    test_size=0.2, random_state=42)
print(f'X_train_wo_corr.shape={X_train_wo_corr.shape},  X_test.shape={X_test_wo_corr.shape}')
print(f'y_train_wo_corr.mean()={y_train_wo_corr.mean()}, y_test.mean()={y_test_wo_corr.mean()}')


# 4. Обучение моделей

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

In [ ]:
def check_model(model, X_train, y_train, X_test, y_test):
    """Оценить качество модели"""
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    mse_test = mean_squared_error(y_test, y_test_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)    
    r2_test = r2_score(y_test, y_test_pred)
    rmse_test = root_mean_squared_error(y_test, y_test_pred)
    
    mse_train = mean_squared_error(y_train, y_train_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)    
    r2_train = r2_score(y_train, y_train_pred)
    rmse_train = root_mean_squared_error(y_train, y_train_pred)
    
    print('Train data:')
    print(f"  MSE:    {round(mse_train,4)}")
    print(f"  RMSE:   {round(rmse_train,4)}")
    print(f"  MAE:    {round(mae_train,4)}")
    print(f"  r2:     {round(r2_train,4)}")
    print(f"  median: {round(y_train.median(),4)}")

    print('Test data:')
    print(f"  MSE:    {round(mse_test,4)}")
    print(f"  RMSE:   {round(rmse_test,4)}")
    print(f"  MAE:    {round(mae_test,4)}")
    print(f"  r2:     {round(r2_test,4)}")    
    print(f"  median: {round(y_test.median(),4)}")
    

## 4.a. statsmodel.OLS

In [ ]:
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS

In [ ]:
def build_OLS_model(X_train, y_train, X_test, y_test, columns):
    X_train_OLS = copy.deepcopy(X_train[columns])
    X_test_OLS = copy.deepcopy(X_test[columns])
    y_train_OLS = copy.deepcopy(y_train)
    y_test_OLS = copy.deepcopy(y_test)    

    X_train_OLS = sm.add_constant(X_train_OLS, prepend=False)
    X_test_OLS = sm.add_constant(X_test_OLS, prepend=False)
    model_OLS = OLS(y_train_OLS, X_train_OLS)
    res_OLS = model_OLS.fit()
    return model_OLS, res_OLS, X_train_OLS, X_test_OLS, y_train_OLS, y_test_OLS

### 4.a.i OLS-модель с оригинальными столбцами

In [ ]:
# построить модель по всем столбцам
columns = X_train.columns.to_list()
model_OLS, res_OLS, X_train_OLS, X_test_OLS, y_train_OLS, y_test_OLS = build_OLS_model(X_train, y_train, X_test, y_test, columns)
# оценим модель
print(res_OLS.summary())

In [ ]:
check_model(res_OLS, X_train_OLS, y_train_OLS, X_test_OLS, y_test_OLS)

p-value высокое у num_children_at_home, gross_weight, recyclable_package, low_fat, units_per_case и store_sales(in millions)_without_outliers 

Надо попробовать исключить их их модели и снова построить модель.

In [ ]:
columns = X_train.columns.to_list()
columns.remove('num_children_at_home')
columns.remove('gross_weight')
columns.remove('recyclable_package')
columns.remove('low_fat')
columns.remove('units_per_case')
columns.remove('store_sales(in millions)_without_outliers')
model_OLS2, res_OLS2, X_train_OLS2, X_test_OLS2, y_train_OLS2, y_test_OLS2 = build_OLS_model(X_train, y_train, X_test, y_test, columns)
# оценим модель
print(res_OLS2.summary())

R-squared не изменилось после удаления столбцов - значи их можно безопасно удалить.

In [ ]:
check_model(res_OLS2, X_train_OLS2, y_train_OLS2, X_test_OLS2, y_test_OLS2)

### 4.a.ii OLS-модель с столбцами без корреляций

In [ ]:
# построить модель по всем столбцам
columns = X_train_wo_corr.columns.to_list()
model_OLS_wo_corr, res_OLS_wo_corr, X_train_OLS_wo_corr, X_test_OLS_wo_corr, y_train_OLS_wo_corr, y_test_OLS_wo_corr = build_OLS_model(X_train_wo_corr, y_train_wo_corr, X_test_wo_corr, y_test_wo_corr, columns)
# оценим модель
print(res_OLS_wo_corr.summary())

In [ ]:
check_model(res_OLS_wo_corr, X_train_OLS_wo_corr, y_train_OLS_wo_corr, X_test_OLS_wo_corr, y_test_OLS_wo_corr)

In [ ]:
columns = X_train_wo_corr.columns.to_list()
columns.remove('num_children_at_home')
columns.remove('gross_weight')
columns.remove('recyclable_package')
columns.remove('low_fat')
columns.remove('units_per_case')
columns.remove('salad_bar')
model_OLS2_wo_corr, res_OLS2_wo_corr, X_train_OLS2_wo_corr, X_test_OLS2_wo_corr, y_train_OLS2_wo_corr, y_test_OLS2_wo_corr = build_OLS_model(X_train_wo_corr, y_train_wo_corr, X_test_wo_corr, y_test_wo_corr, columns)
# оценим модель
print(res_OLS2_wo_corr.summary())

In [ ]:
check_model(res_OLS2_wo_corr, X_train_OLS2_wo_corr, y_train_OLS2_wo_corr, X_test_OLS2_wo_corr, y_test_OLS2_wo_corr)

## 4.b. scikit-learn.LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
def build_LR_model(X_train, y_train, X_test, y_test, columns):
    """Построить LinearRegression-модель"""
    X_train_LR = copy.deepcopy(X_train[columns])
    X_test_LR = copy.deepcopy(X_test[columns])
    y_train_LR = copy.deepcopy(y_train)
    y_test_LR = copy.deepcopy(y_test)    

    model_LR = LinearRegression()
    res_LR = model_LR.fit(X_train_LR, y_train_LR)
    return model_LR, res_LR, X_train_LR, X_test_LR, y_train_LR, y_test_LR

def show_LR_coef(model_LR):
    print('Коэффициенты модели:')
    for i in range(len(model_LR.feature_names_in_)):
        print(f'  {model_LR.feature_names_in_[i]}={round(model_LR.coef_[i],4)}')
    print(f'  const={round(model_LR.intercept_, 4)}')

### 4.b.i LR-модель с оригинальными столбцами

In [ ]:

columns = X_train.columns.to_list()
model_LR, res_LR, X_train_LR, X_test_LR, y_train_LR, y_test_LR = build_LR_model(X_train, y_train, X_test, y_test, columns)
check_model(res_LR, X_train_LR, y_train, X_test_LR, y_test_LR)
show_LR_coef(res_LR)

### 4.b.ii LR-модель со столбцами без корреляции

In [ ]:
columns = X_train_wo_corr.columns.to_list()
model_LR_wo_corr, res_LR_wo_corr, X_train_LR_wo_corr, X_test_LR_wo_corr, y_train_LR_wo_corr, y_test_LR_wo_corr = build_LR_model(X_train_wo_corr, y_train_wo_corr, X_test_wo_corr, y_test_wo_corr, columns)
check_model(res_LR_wo_corr, X_train_LR_wo_corr, y_train_wo_corr, X_test_LR_wo_corr, y_test_LR_wo_corr)
show_LR_coef(res_LR_wo_corr)

# Вывод
Оба типа моделей (OLS и LR) показывают сопоставимые результаты.
Удаление стоблцов с корреляцией как минимум не улучшает (а скорее - ухудшает) качество модели - метрики MSE, RMSE, MAE остаются практически теми же, при этом метрика R-squared ухудшается.

В целом, получившиеся модели малополезны дял предсказания, т.к.: 
* RMSE сопоставимо с медианным значением целевой переменной
* R-squared мало - 0.02 в лучшем случае.
